In [1]:
# @title Install the SDK
!pip install -q google-genai

In [2]:
# @title Step 1: Authentication
# Set your API key here.
import os
from google.colab import userdata

# Option 1: Using Colab Secrets (Recommended)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Option 2: Hardcoded (For testing only)
#os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE"

if "GOOGLE_API_KEY" not in os.environ:
    print("⚠️ Warning: Please set your GOOGLE_API_KEY before continuing.")

In [3]:
# @title Step 2: Define Function Declarations

from google import genai
from google.genai import types

# 1. Tool: Get Order Details
get_order_tool = types.FunctionDeclaration(
    name="get_order_details",
    description="Retrieves order status and purchase date. Required before any refund.",
    parameters=types.Schema(
        type="OBJECT",
        properties={
            "order_id": types.Schema(type="STRING")
        },
        required=["order_id"]
    )
)

# 2. Tool: Process Refund
# Note: The model should ONLY call this if the order is eligible.
process_refund_tool = types.FunctionDeclaration(
    name="process_refund",
    description="Submits a refund request. Only use if order is within 30 days.",
    parameters=types.Schema(
        type="OBJECT",
        properties={
            "order_id": types.Schema(type="STRING"),
            "reason": types.Schema(type="STRING")
        },
        required=["order_id", "reason"]
    )
)

tools_conf = [types.Tool(function_declarations=[get_order_tool, process_refund_tool])]
client = genai.Client()
print("✅ Tools defined: get_order_details, process_refund")

✅ Tools defined: get_order_details, process_refund


In [10]:
# @title Step 3: The "Happy Path" (Policy Success)

import datetime

# Let's say today is 2024-11-24. We'll simulate an order from 10 days ago.
today_date = "2024-11-24"
simulated_order_date = "2024-11-14"

print(f"📅 Today: {today_date}")
print(f"📦 Order Date: {simulated_order_date} (Eligible for return)")
print("-" * 40)

# 1. Start Conversation
history = [
    types.Content(
        role="user",
        parts=[types.Part.from_text(text = f"Today is {today_date}. I want to return Order #999 because it's broken.")]
    )
]

# 2. First Turn: Model should ask for Order Details
resp1 = client.models.generate_content(
    model='gemini-3-pro-preview',
    contents=history,
    config=types.GenerateContentConfig(tools=tools_conf)
)

fc_1 = resp1.candidates[0].content.parts[0].function_call
print(f"1️⃣ Model requests: {fc_1.name} (args: {fc_1.args})")

# --- SAVE STATE (The Golden Rule) ---
# We save the model's question + its hidden signature
history.append(resp1.candidates[0].content)

# 3. Simulate Client Execution
# We return the data that makes the order ELIGIBLE
tool_output = {
    "order_id": "#999",
    "purchase_date": simulated_order_date,
    "status": "delivered"
}

history.append(types.Content(
    role="tool",
    parts=[types.Part.from_function_response(
        name=fc_1.name,
        response={"content": tool_output}
    )]
))

# 4. Second Turn: Model validates logic -> Calls Refund
print("2️⃣ Sending order details back...")
resp2 = client.models.generate_content(
    model='gemini-3-pro-preview',
    contents=history,
    config=types.GenerateContentConfig(tools=tools_conf)
)

fc_2 = resp2.candidates[0].content.parts[0].function_call

if fc_2 and fc_2.name == "process_refund":
    print(f"✅ SUCCESS! Model verified date ({simulated_order_date}) is < 30 days.")
    print(f"   Action: {fc_2.name}")
    print(f"   Signature Present? {'✅ Yes' if resp2.candidates[0].content.parts[0].thought_signature else '❌ No'}")
else:
    print(f"❌ Failed or denied: {resp2.text}")

📅 Today: 2024-11-24
📦 Order Date: 2024-11-14 (Eligible for return)
----------------------------------------
1️⃣ Model requests: get_order_details (args: {'order_id': '999'})
2️⃣ Sending order details back...
✅ SUCCESS! Model verified date (2024-11-14) is < 30 days.
   Action: process_refund
   Signature Present? ✅ Yes


In [6]:
# @title Step 4: The "Failure Path" (Amnesia)
print("\n💥 STARTING FAILURE DEMO 💥")

# 1. Reset
history_bad = [
    types.Content(role="user", parts=[types.Part.from_text(text=f"Today is {today_date}. Return Order #999.")])
]

# 2. First Call
resp1 = client.models.generate_content(
    model='gemini-3-pro-preview',
    contents=history_bad,
    config=types.GenerateContentConfig(tools=tools_conf)
)

fc_1 = resp1.candidates[0].content.parts[0].function_call
print(f"1️⃣ Model requests: {fc_1.name}")

# --- 🚨 THE BUG: Manual Reconstruction without Signature ---
bad_part = types.Part.from_function_call(name=fc_1.name, args=fc_1.args)
# Note: bad_part.thought_signature is None!

history_bad.append(types.Content(role="model", parts=[bad_part]))

# 3. Return the SAME eligible data
tool_output = {"order_id": "#999", "purchase_date": simulated_order_date, "status": "delivered"}

history_bad.append(types.Content(
    role="tool",
    parts=[types.Part.from_function_response(name=fc_1.name, response={"content": tool_output})]
))

# 4. Second Call (Should Fail)
print("2️⃣ Sending details back (Signature Omitted)...")

try:
    client.models.generate_content(
        model='gemini-3-pro-preview',
        contents=history_bad,
        config=types.GenerateContentConfig(tools=tools_conf)
    )
    print("❌ Unexpected Success (API should have rejected this)")
except Exception as e:
    print("✅ CRASH SUCCESSFUL! API rejected the request.")
    print(f"   Error: {e}")



💥 STARTING FAILURE DEMO 💥
1️⃣ Model requests: get_order_details
2️⃣ Sending details back (Signature Omitted)...
✅ CRASH SUCCESSFUL! API rejected the request.
   Error: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Function call is missing a thought_signature in functionCall parts. This is required for tools to work correctly, and missing thought_signature may lead to degraded model performance. Additional data, function call `default_api:get_order_details` , position 2. Please refer to https://ai.google.dev/gemini-api/docs/thought-signatures for more details.', 'status': 'INVALID_ARGUMENT'}}


Let's break down exactly what the initial thought signature represents in the context of the E-Commerce Refund Agent.

The Scenario's First Step

User: "Today is 2024-11-24. I want to return Order #999 because it's broken."

Model's First Action: It doesn't just immediately say get_order_details("#999"). Before it generates that function call, it does a "thinking step."

What the Model is "Thinking" (The Logic)

When the model receives that prompt, it internally runs a logic process that looks something like this:

Analyze User Intent: The user wants to return an item.

Identify Constraints: The user provided an Order ID (#999) and a reason ("broken"). They also provided today's date.

Consult Tool Definitions (Policy Check): The model looks at its available tools. It sees process_refund and reads its description: "Submits a refund request. Only use if order is within 30 days."

Formulate a Plan:

Goal: Process the refund.

Problem: I don't know if the order is within 30 days. I don't have the purchase date.

Solution: I must first get the order details.

Action: Call get_order_details(order_id="#999").

Set a Mental Reminder (The Crucial Part): "I am calling get_order_details. When I get the result, I must compare the purchase_date against today's date (2024-11-24). If the difference is <= 30 days, I will then proceed to call process_refund with the reason 'broken'. If it's > 30 days, I will deny the request."

What the Initial Signature Represents

The first thought signature, let's call it <Sig_A>, is an encrypted snapshot of Step 5 (The Mental Reminder).

It doesn't just say "I called a function." It cryptographically encodes:

The Pending Plan: "I am in the middle of a multi-step refund workflow."

The User's Inputs: "Order #999", "Reason: broken", "Today: 2024-11-24".

The Conditional Logic to Execute Next: "The next step depends on a date comparison. I need to remember to perform that comparison once I have the data."

Why it Fails Without it

In the failure demo, we did this:

User asks for refund.

Model sends get_order_details("#999") + <Sig_A>.

We send back the result: {"purchase_date": "2024-11-14"}.

BUT we throw away <Sig_A>.

When the model receives the purchase date without the signature, its "mind" is blank. It has forgotten the plan. It sees:

Input: {"purchase_date": "2024-11-14"}

Context: None. (It doesn't remember the user asked for a refund).

It's like handing a cashier a receipt and saying nothing. They will look at you and say, "Okay, what do you want me to do with this?"

The model, having lost the "Refund Plan" held in the signature, doesn't know it's supposed to check the date against the 30-day policy. It might just say, "Here are the details for order #999," completely failing to process the refund.

In short: The initial signature holds the plan and the conditional logic that needs to be executed after the tool returns its data